# **PROCEDIMIENTO DE REVISIÓN DEL CÓDIGO** #

### Primera parte

El primer paso para llevar a cabo este proceso fue revisar la factorización PLU, primeramente para matrices chicas (de hasta 10^3 x 10^3). Para ello el primer programa que realizamos para detectar errores, fue el de **"revision_factorizacion_PLU.py"** (se encuentra en la carpeta de /notebooks/Revision/Historicos) en el cual se encuentra la función "revision_PLU" y cuyo algoritmo es el siguiente:

1) Genera matrices aleatorias de diferentes dimensiones y entradas.<br>
2) Intenta calcular la inversa de la matriz A generada aleatoriamente, para que el producto con ésta regrese la matriz identidad y así garantizar que sea no singular.<br>
3) Realiza la factorización PLU con el algoritmo realizado por el equipo de programación.<br>
4) Compara los resultados del algoritmo desarrollado, cuyo output es **P,L,U** con las que se obtienen del paquete "scipy" con una tolerancia absoluta de 10^-05.<br>
5) Si la condición de la comparación del paso anterior es "verdadera" se arroja un status "Correcto", si esta condición no se cumple devuelve un archivo con la matriz A y las matrices PLU de ambos algoritmos. El archivo mencionado se ocupó para levantar algunos issues.<br>
6) Por último, devuelve un dataframe con las siguientes variables: dimensión, status_plu, tiempo_plu (lo que tardó en correr la función para obtener las matrices PLU) y matrices_incorrectas.<br>

In [ ]:
import sys
sys.path.append('./../../')

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

#Own Library modules
import src
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

#Funciones de revisión
from src.test_algorithms import revision_factorizacion_PLU

Los argumentos que recibe la primera función **revision_PLU** :<br>
* Nombre del archivo, en este caso "resultados.txt"<br>
* Número de corridas que se refiere a cuantas veces corrimos el algoritmo para hacer la revisión, en este caso 15<br>
* Límite inferior de la dimensión de la matriz, en este caso 10^2<br>
* Límite superior de la dimensiónd de la matriz, en este caso 10^2+1<br>
* Límite inferior de las entradas de A, es decir el valor de cada uno de los aij que puede tener la matriz, en este caso -99.<br>
* Límite superior de las entradas de A, es decir el valor de cada uno de los aij que puede tener la matriz, en este caso 99.<br>

In [11]:
revision_factorizacion_PLU.revision_PLU('resultados.txt',15,10**2,10**2+1,-99,99)

,dimension,status_plu,tiempo_plu,matrices_incorrectas
0,100,Incorrecto,2.166639,P
1,100,Incorrecto,1.791332,P
2,100,Incorrecto,2.083854,P
3,100,Incorrecto,1.797055,P
4,100,Incorrecto,1.114240,P
5,100,Incorrecto,1.779011,P
6,100,Incorrecto,1.264601,P
7,100,Incorrecto,1.596583,P
8,100,Incorrecto,1.181513,P
9,100,Incorrecto,1.489248,P


#### Comentarios
<i>Al ejecutar repetidas veces este programa, detectamos que la matriz que siempre salía con status incorrecto era la matriz de permutación (P) que, al revisar la documentación de "scipy", observamos que A=PLU, mientras que en nuestro algoritmo se cumple que PA=LU, por lo que P en ambos algoritmos no iban a ser iguales.</i>

### Segunda parte

Una vez detectado lo anterior, se decidió ya no usar el paquete "scipy" y sólo validar haciendo que PA=LU. 

Por lo que se realizó otro script llamado **"revision_modular.py"** en el cual, se encuentran principalmente las funciones de **revision_PLU**, **revision_x** y **revision_bloques**.

En la de revision_PLU se hace básicamente lo descrito en la primera parte de la revisión, pero se eliminó lo siguiente:<br>

1) Genera matrices aleatorias de diferentes dimensiones y entradas.<br>
<del>2) Intenta calcular la inversa de calcular su inversa de la matriz A generada aleatoriamente, para que el producto con ésta regrese la matriz identidad y así garantizar que sea no singular.</del>->Este paso se eliminó porque se implementó en el código que realiza la factorización PLU.<br>
3) Realiza la factorización PLU con el algoritmo realizado por el equipo de programación.<br>
<del>4) Compara los resultados del algoritmo desarrollado, cuyo output es **P,L,U** con las que se obtienen del paquete "scipy" con una tolerancia absoluta de 10^-05.</del>->Este paso se modificó a comprobar que PA=LU por la conclusión de la primera revisión.<br>
5) Si la condición de la comparación del paso anterior es "verdadera" se arroja un status "Correcto",<del> si esta condición no se cumple devuelve un archivo con la matriz A y las matrices PLU de ambos algoritmos.</del>.<br>
6) Por último, devuelve un dataframe con las siguientes variables: dimensión, **el número de condición de la matriz A**, tiempo_plu (lo que tardó en correr la función para obtener las matrices PLU) y el estatus del algoritmo.<br>


In [13]:
from src.test_algorithms import revision_modular

Los argurmentos que recibe la función **revision_PLU**:<br>
* Número de corridas que se refiere a cuantas veces corrimos el algoritmo para hacer la revisión, en este caso 3.<br>
* Límite inferior de la dimensión de la matriz, en este caso 10^3<br>
* Límite superior de la dimensiónd de la matriz, en este caso 10^3+1<br>
* Límite inferior de las entradas de A, es decir el valor de cada uno de los aij que puede tener la matriz, en este caso -99.<br>
* Límite superior de las entradas de A, es decir el valor de cada uno de los aij que puede tener la matriz, en este caso 99.<br>

In [ ]:
revision_modular.revision_PLU(3,10**3,10**3+1,-99,99)

#### Comentarios
<i>Al hacer varias corridas detectamos que el programa estaba arrojando cálculos correctos.</i>

### Tercera parte

Lo siguiente fue validar el valor de x con la función "revision_x" y cuyo algoritmo es el siguiente:<br>
1) Genera matrices aleatorias de diferentes dimensiones y entradas.<br>
2) Se calcula el número de condición de la matriz A generada en el punto anterior.<br>
3) Se genera aleatoriamente un vector x ("real")<br>
4) Se multiplica el vector x ("real") por la matriz A para obtener el lado derecho del sistema de ecuaciones lineales (b)<br>
5) Se ejecuta el algoritmo de programación para resolver el sistema de ecuaciones $AX=b$ y obtener la x estimada.<br>
6) Se comparan las x's con una tolerancia absoluta de 10^-05 y sí se cumple esto se arroja un status "Correcto" y se obtiene el promedio del error absoluto de ambos vectores.<br>
7) Se imprime un dataframe con las siguientes variables: dimension_A, condicion_A, tiempo_x (lo que tardó en correr la función para obtener el valor de x), estado_x, error_absoluto y tipo de matriz.

Los argurmentos que recibe la función **revision_x** :<br>
* Número de corridas que se refiere a cuantas veces corrimos el algoritmo para hacer la revisión, en este caso 3<br>
* Límite inferior de la dimensión de la matriz, en este caso 10^3<br>
* Límite superior de la dimensión de la matriz, en este caso 10^3+1<br>
* Límite inferior de las entradas de A, es decir el valor de cada uno de los aij que puede tener la matriz, en este caso -99.<br>
* Límite superior de las entradas de A, es decir el valor de cada uno de los aij que puede tener la matriz, en este caso 99.<br>

In [ ]:
revision_modular.revision_x(3,10**3,10**3+1,-99,99)

#### Comentarios
<i>Al hacer varias corridas detectamos que el programa estaba arrojando cálculos correctos para la X estimada en comparación con la X "real"</i>

### Cuarta parte

Finalmente pasamos a la revisión del algoritmo por bloques "solve_blocks", el cual integra el algoritmo de la factorización PLU (previamente revisado), la solución de X por bloques fue revisada con la funcion **"revision_bloques"** y el algoritmo que sigue esta función es el siguiente:<br>
1) Genera matrices aleatorias de diferentes dimensiones y entradas.<br>
2) Genera el vector de X's aleatorio de dimensión n.<br>
3) Se calcula B con la operación $AX=B$.<br>
4) Se calcula el número de condición de la matriz A generada.<br>
5) Se ejecuta el algoritmo de programación para resolver X por bloques.<br>
6) Se calcula el error absoluto entre la <i>Xestimada</i> y <i>Xreal</i>, para posteriormente calcular el promedio de esos errores absolutos.<br>
7) Se comparan las X's con una tolerancia absoluta de 10^-05 y sí se cumple esto se arroja un status "Correcto".<br>
8) Se imprime un dataframe con las siguientes variables: dimension_A, condicion_A, tiempo_bloques (lo que tardó en correr la función por bloques para resolver X), el error_absoluto y tipo de matriz.

In [ ]:
bloques=revision_modular.revision_bloques(5,10**2,10**3+1,-99,99)

Los argurmentos que recibe la función **revision_bloques** :<br>
* Número de corridas que se refiere a cuantas veces corrimos el algoritmo para hacer la revisión, en este caso 5<br>
* Límite inferior de la dimensión de la matriz, en este caso 10^2<br>
* Límite superior de la dimensión de la matriz, en este caso 10^3+1<br>
* Límite inferior de las entradas de A, es decir el valor de cada uno de los aij que puede tener la matriz, en este caso -99.<br>
* Límite superior de las entradas de A, es decir el valor de cada uno de los aij que puede tener la matriz, en este caso 99.<br>

In [ ]:
bloques

In [ ]:
bloques2=revision_modular.revision_bloques(3,10**3,10**3+1,-99,99)

In [ ]:
bloques2

In [ ]:
bloques.sort_values(by=['dimension_A'],inplace=True)
dimension = bloques['dimension_A']
tiempo=bloques['tiempo_bloques']

In [ ]:
import matplotlib.pyplot as plt
plt.plot(tiempo,dimension,'o-')
plt.title('Gráfica dimension de matriz contra el tiempo de ejecución')
plt.xlabel('dimension')
plt.ylabel('tiempo_bloques')
plt.grid()
plt.show()

#### Comentarios
<i>Al hacer varias corridas detectamos que el programa estaba arrojando cálculos correctos para la X estimada con el algoritmo de bloques, y de igual forma obtuvimos que el error absoluto es en casi todos los casos de orden 10^-10 lo cual indica que nuestro algoritmo tiene una buena aproximación. 
    
Además, hicimos una gráfica de la dimensión de la matriz contra el tiempo de ejecución del algoritmo y se obtiene que a mayor dimensión de la matriz generada mayor tiempo de ejecución, lo cual era lo esperado.</i>

**Sin embargo, es importante señalar que el tiempo de ejecución baja considerablemente para matrices de 10^3 x 10^3 usando el algoritmo por bloques.**

Para el caso de matrices singulares, el algoritmo por bloques arroja el siguiente mensaje:

In [ ]:
import numpy as np

In [ ]:
#definimos matriz singular y su lado derecho
A=np.array([[1,2],
          [1,2]])
b=np.array([3,3])

In [ ]:
revision_modular.resuelve_bloques(A,b)